In [1]:
# This is a programm created to observe current headcount from 'internal' roster (ggsheet) and workplan and client requirement.
# Here we will also compare them for the mismatches and try to make adjustment ASAP

In [2]:
# Libraries
import os
import pandas as pd
from datetime import datetime as dt
from datetime import time as t
from dateutil import relativedelta
import numpy as np
from IPython.display import display

In [3]:
# IMPORT USER CREDENTIAL HERE. UPDATE THIS SO THAT THE CODE WILL BE ABLE TO RUN WITH EACH SPECIFIC USER
user_credential = r'/Users/dinhhoang.nguyen.CONCENTRIX/OneDrive - Concentrix Corporation\WFM-internal'

In [4]:
# IMPORT MASTER ROSTER
masterroster = pd.read_json(os.path.join(r'C:', user_credential, r'DB\filejson\masterroster.json'))
masterroster = masterroster[['Employee_ID', 'LWD', 'Status']]
masterroster['LWD'] = pd.to_datetime(masterroster['LWD'], format='mixed').dt.strftime('%Y-%m-%d')

In [5]:
# IMPORT SCHEDULE
schedule = pd.read_json(os.path.join(r'C:', user_credential, r'DB\filejson\schedule.json'))
schedule['Week'] = pd.to_datetime(schedule['Date'], format='mixed').dt.strftime('%Y%W')

In [6]:
# IMPORT WORKPLAN
workplan = pd.read_json(os.path.join(r'C:', user_credential, r'DB\filejson\workplan_merge_iex.json'))
# Create columns to aggregate Agents'
productive_cond = (workplan['Scheduled Activity']=='Email 1')|(workplan['Scheduled Activity']=='Open Time')
downtime_cond = (workplan['Scheduled Activity']=='Team Meeting')|(workplan['Scheduled Activity']=='Coaching 1:1')
IOS_cond = (workplan['Scheduled Activity']=='Break')|(workplan['Scheduled Activity']=='Break Offline')
training_cond = (workplan['Scheduled Activity']=='Training')|(workplan['Scheduled Activity']=='Training Offline')
lunch_cond = (workplan['Scheduled Activity']=='Lunch')
# Summarize
workplan['productive'] = workplan.apply(lambda x: x['Length'] if (x['Scheduled Activity']=='Email 1')|(x['Scheduled Activity']=='Open Time') else 0, axis=1)*24
workplan['downtime'] = workplan.apply(lambda x: x['Length'] if (x['Scheduled Activity']=='Team Meeting')|(x['Scheduled Activity']=='Coaching 1:1') else 0, axis=1)*24
workplan['IOS'] = workplan.apply(lambda x: x['Length'] if (x['Scheduled Activity']=='Break')|(x['Scheduled Activity']=='Break Offline') else 0, axis=1)*24
workplan['training'] = workplan.apply(lambda x: x['Length'] if (x['Scheduled Activity']=='Training')|(x['Scheduled Activity']=='Training Offline') else 0, axis=1)*24
workplan['lunch'] = workplan.apply(lambda x: x['Length'] if (x['Scheduled Activity']=='Lunch') else 0, axis=1)*24
# Add staffed_time
func_cond = {'productive': 'sum', 'downtime': 'sum', 'IOS': 'sum', 'training': 'sum', 'lunch': 'sum'}
workplan = workplan.groupby(['EID', 'Date', 'LOB'], as_index=False).agg(func_cond)
workplan['staffed_time'] = round(workplan['productive'] + workplan['downtime'] + workplan['IOS'] + workplan['training'] + workplan['lunch'], 3)

In [7]:
# Merge schedule and Workplan
main_frame = pd.merge(schedule, workplan, left_on=['Emp ID', 'Date'], right_on=['EID', 'Date'], how='left')
main_frame[['productive', 'downtime', 'IOS', 'training', 'lunch']] = main_frame[['productive', 'downtime', 'IOS', 'training', 'lunch']].fillna(0)
main_frame = main_frame.rename(columns={'LOB_x': 'LOB_Schedule', 'LOB_y': 'LOB_IEX'})

In [8]:
# Create columns for validations
## check if an agent is available in internal roster but missing in workplan
### id_exist is a data frame to check if the agent is missing in a specific date or the whole week
id_exist = main_frame[['Week', 'Emp ID', 'EID']].drop_duplicates().groupby(['Week', 'Emp ID'], as_index=False)['EID'].count() #EID will show 1 if any ID in a week, otherwise, it would return 0
id_exist = id_exist.rename(columns={'EID': 'id_exist'})
main_frame = pd.merge(main_frame, id_exist, on=['Week', 'Emp ID'], how='left')
def iex_missing(x):
    if x['Shift'] != 'OFF' and pd.isnull(x['EID']):
        if x['id_exist'] == 1:
            return 'date_missing'
        else:
            return 'week_missing'
    else:
        return 'valid'
main_frame['iex_missing_name'] = main_frame.apply(iex_missing, axis=1) #we can now categorize if the agent ID is not missing, missing in a date, or the whole week
### now we need to explain why the agent is missing
### we get the LWD from master roster to check if the agent is still active not not
main_frame = pd.merge(main_frame, masterroster, left_on='Emp ID', right_on='Employee_ID', how='left')
main_frame = main_frame.drop(columns={'Employee_ID'})

In [9]:
# we check if total working time of an agent is 9 in a day or not, if not, we may need to double check again with JP
main_frame['staffed_time_missing'] = main_frame.apply(lambda x: 'mismatch' if not(pd.isnull(x['EID'])) and x['staffed_time'] != 9 else 'valid', axis=1)
# if the other two is valid, then we will continue to validate if the LOB in the internal roster and workplan is not mismatched
main_frame['lob_mismatch'] = main_frame.apply(lambda x: 'mismatch' if not(pd.isnull(x['EID']))and x['LOB_Schedule'] != x['LOB_IEX'] else 'valid', axis=1)

In [10]:
# Check attrition in workplan
workplan['Week'] = pd.to_datetime(workplan['Date'], format='mixed').dt.strftime('%Y-%m')
workplan_attrition = workplan
workplan_attrition = pd.merge(workplan_attrition, masterroster, left_on='EID', right_on='Employee_ID', how='left')
workplan_attrition['invalid_plan'] = (pd.to_datetime(workplan_attrition['LWD'], format='mixed') - workplan_attrition['Date']).dt.days

In [11]:
workplan_attrition['invalid_plan'] = (pd.to_datetime(workplan_attrition['LWD'], format='mixed') - workplan_attrition['Date']).dt.days


In [14]:
# Export data to excel
path_1 = os.path.join(r'C:', user_credential, 'Dispatch files\Roster validation\Roster Validation.xlsx')

with pd.ExcelWriter(path_1, mode="a", engine="openpyxl",if_sheet_exists="replace") as writer:
    main_frame.to_excel(writer, sheet_name='Sheet1', index=False)
    workplan_attrition.to_excel(writer, sheet_name='Sheet2', index=False)